# DL Competition 4 Report
The final report should contain following points:
1. Describe what you have done to improve your training accuracy in detail.
2. Explain your code in your notebook for each block.
3. Your training script. We will make sure that your results are reproducible.